<a href="https://www.kaggle.com/code/siddp6/simple-skip-connection?scriptVersionId=138506041" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Imports

In [ ]:
import utility

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Initalisation

In [ ]:
input_size, output_size = 1024, 8
num_samples = 1000
epochs = 100
input_data = torch.rand(num_samples, input_size)
target_data = torch.rand(num_samples, output_size)
loss_normal_3, loss_normal_6 = list(), list()
loss_skip_3, loss_skip_6 = list(), list()

# Normal Model (3 layered) (without skip-connection)

In [ ]:
class NormalNet3(nn.Module):
    def __init__(self, input_size, output_size):
        super(NormalNet3, self).__init__()
        
        self.layer_1 = nn.Linear(input_size, 128)
        self.layer_2 = nn.Linear(128, 16)
        self.layer_3 = nn.Linear(16, output_size)
        
    def forward(self, X):
        X = F.relu(self.layer_1(X))
        X = F.relu(self.layer_2(X))
        X = self.layer_3(X)
        
        return X

In [ ]:
normal_model3 = NormalNet3(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(normal_model3.parameters(), lr=.01)

In [ ]:
loss_normal_3 = utility.train_model(normal_model3, epochs, input_data, target_data, criterion, optimizer, 5)

# Normal Model (7 layered) (without skip-connection)

In [ ]:
class NormalNet7(nn.Module):
    def __init__(self, input_size, output_size):
        super(NormalNet7, self).__init__()
        
        self.layer_1 = nn.Linear(input_size, 512)        
        self.layer_2 = nn.Linear(512, 256)
        self.layer_3 = nn.Linear(256, 128)
        self.layer_4 = nn.Linear(128, 64)
        self.layer_5 = nn.Linear(64, 32)
        self.layer_6 = nn.Linear(32, 16)
        self.layer_7 = nn.Linear(16, output_size)
        
    def forward(self, X):
        X = F.relu(self.layer_1(X))
        X = F.relu(self.layer_2(X))
        X = F.relu(self.layer_3(X)) 
        X = F.relu(self.layer_4(X)) 
        X = F.relu(self.layer_5(X)) 
        X = F.relu(self.layer_6(X)) 
        X = self.layer_7(X)
        
        return X

In [ ]:
normal_model7 = NormalNet7(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(normal_model7.parameters(), lr=.01)

In [ ]:
loss_normal_7 = utility.train_model(normal_model7, epochs, input_data, target_data, criterion, optimizer, 5)

# Skip Model(with skip-connection)

In [ ]:
class SkipNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(SkipNN, self).__init__()
        
        self.layer_1 = nn.Linear(input_size, 512)        
        self.layer_2 = nn.Linear(512, 256)
        self.layer_3 = nn.Linear(256, 128)
        self.layer_4 = nn.Linear(128, 64)
        self.layer_5 = nn.Linear(64, 32)
        self.layer_6 = nn.Linear(32, 16)
        self.layer_7 = nn.Linear(16, output_size)
        
    def forward(self, X):
        X1 = F.relu(self.layer_1(X))
        X2 = F.relu(self.layer_2(X1)) + X1
        X3 = F.relu(self.layer_3(X2)) + X1
        X4 = F.relu(self.layer_4(X3)) + X1
        X5 = F.relu(self.layer_5(X4)) + X1
        X6 = F.relu(self.layer_6(X5)) + X1
        X7 = self.layer_7(X6)
        
        return X7

In [ ]:
skip_model = NormalNet(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(skip_model.parameters(), lr=.01)

In [ ]:
loss_skip = utility.train_model(skip_model, epochs, input_data, target_data, criterion, optimizer, 5)

# Analysis

In [ ]:
utility.plot_loss({"Normal": loss_normal, "Skip": loss_skip}, epochs)